In [ ]:
import sympy as sym
from sympy.printing.ccode import C99CodePrinter
from sympy.core.function import UndefinedFunction

class SubCodePrinter(C99CodePrinter):
    """Print derivative of a function of symbols in a shorter form.
    """
    def _print_Derivative(self, expr):
        function, *vars = expr.args
        number_of_derivatives = len(expr.args) - 1
        if function.func.__name__[0:2] == 'mu':
            function_string_index = (
                int(sym.srepr(function).split("'")[1][2:]) - 1)
            if number_of_derivatives == 1:
                derivative_string = repr(vars[0][0])
                derivative_order  = '' if vars[0][1] == 1 else str(vars[0][1])
                result = ('(*endmember[' + str(function_string_index) + '].d' 
                          + derivative_order + 'mu0d' 
                          + derivative_string + derivative_order + ')(T, P)')
            elif number_of_derivatives == 2:
                derivative_string_2 = repr(vars[0][0])
                derivative_order_2  = '' if vars[0][1] == 1 else str(vars[0][1])
                derivative_string_1 = repr(vars[1][0])
                derivative_order_1  = '' if vars[1][1] == 1 else str(vars[1][1])
                derivative_total    = str(vars[0][1]+vars[1][1])
                result = ('(*endmember[' + str(function_string_index) + '].d' 
                          + derivative_total + 'mu0d' 
                          + derivative_string_1 + derivative_order_1 +'d' 
                          + derivative_string_2 + derivative_order_2 + ')(T, P)')
            else:
                result = ''
        
        else:
            if (not isinstance(type(function), UndefinedFunction) or 
                not all(isinstance(i, Symbol) for i in vars)):
                return super()._print_Derivative(expr)
        return result


printer = SubCodePrinter(settings={"user_functions":{"Q":"born_Q"}})

T = sym.Symbol("T")
P = sym.Symbol("P")
f = sym.Function("Q")(T,P)
expr = f.diff(T,T,P)

# Print the expression using the normal latex printer and our custom
# printer.
printer.doprint(f)